### Data Prep

In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import skimage.io
import datetime
import random
from scipy.signal import spectrogram
#import tensorflow as tf

In [2]:
# File paths
TESTDIR = './../Data/Hyundai/'
TOYOTALC = './../Data/ToyotaLc/'
GENERIC = './../Data/GenericFsk/'
TESTPATH = 'Lock/'
TESTSIG = 'keyfob_signal2023Y1022163258.raw'

TESTOUT = './../Data/MelSpecV1/Hyundai/'
LCOUT = './../Data/MelSpecV1/ToyotaLc/'

In [3]:
IMG_SIZE=640
IMG_L = 640

Separate I from Q
- we only care about the "real" portion

Trim our samples to get rid of the noise from start and end of capture

In [4]:
"""# Specify the path to your .raw file
data_dir = './../Data/Hyundai/Unlock/'

# Get a list of all .raw files in the directory
raw_files = [file for file in os.listdir(data_dir) if file.endswith(".raw")]

# Loop through each .raw file in the directory
for raw_file in raw_files:
    file_path = os.path.join(data_dir, raw_file)

    # Read the raw file as bytes
    with open(file_path, 'rb') as file:
        raw_data = file.read()

    # Convert the raw data to a NumPy array of complex numbers
    # Assuming the raw data consists of interleaved I and Q samples (16-bit signed integers)
    raw_array = np.frombuffer(raw_data, dtype=np.int16).astype(np.complex64)

    # Extract the in-phase (I) and quadrature (Q) components
    I = raw_array.real
    Q = raw_array.imag

    # Create a time axis for the signal (assuming a fixed sample rate)
    sample_rate = 2e6  # Replace with your actual sample rate
    time = np.arange(0, len(I)) / sample_rate

    # Define the trim duration in seconds
    trim_start = 0.005  # 5 ms
    trim_end = 0.005    # 5 ms

    # Calculate the number of samples to trim
    trim_samples = int(sample_rate * (trim_start + trim_end))

    # Trim the signal
    I_trimmed = I[trim_samples:-trim_samples]
    Q_trimmed = Q[trim_samples:-trim_samples]
    time_trimmed = time[trim_samples:-trim_samples]

    # Plot the I and Q components
    plt.figure(figsize=(12, 6))
    plt.suptitle(f'File: {raw_file}')
    plt.subplot(2, 1, 1)
    plt.plot(time_trimmed, I_trimmed)
    plt.title('In-phase Component (Trimmed)')
    plt.xlabel('Time (s)')
    plt.grid()"""

    #plt.subplot(2, 1, 2)
    #plt.plot(time_trimmed, Q_trimmed)
    #plt.title('Quadrature Component (Trimmed)')
    #plt.xlabel('Time (s)')
    #plt.grid()"""

    #plt.tight_layout()
    #plt.show()

'# Specify the path to your .raw file\ndata_dir = \'./../Data/Hyundai/Unlock/\'\n\n# Get a list of all .raw files in the directory\nraw_files = [file for file in os.listdir(data_dir) if file.endswith(".raw")]\n\n# Loop through each .raw file in the directory\nfor raw_file in raw_files:\n    file_path = os.path.join(data_dir, raw_file)\n\n    # Read the raw file as bytes\n    with open(file_path, \'rb\') as file:\n        raw_data = file.read()\n\n    # Convert the raw data to a NumPy array of complex numbers\n    # Assuming the raw data consists of interleaved I and Q samples (16-bit signed integers)\n    raw_array = np.frombuffer(raw_data, dtype=np.int16).astype(np.complex64)\n\n    # Extract the in-phase (I) and quadrature (Q) components\n    I = raw_array.real\n    Q = raw_array.imag\n\n    # Create a time axis for the signal (assuming a fixed sample rate)\n    sample_rate = 2e6  # Replace with your actual sample rate\n    time = np.arange(0, len(I)) / sample_rate\n\n    # Define th

Normalize our Data

In [5]:
# Create a Min-Max scaler
scaler = MinMaxScaler()

# Fit and transform the data (I)
#I_scaled = scaler.fit_transform(I_trimmed.reshape(-1, 1))

Split our data

In [6]:
sample_rate = 8e6

FOR IMAGE CNN VERSION ONLY

In [7]:
# Directory path containing your RF signal files
directory_path = TESTDIR+TESTPATH

center_freq = 433.9e6

# List categories (e.g., Category1, Category2)
categories = ['Lock', 'Unlock','None']

for category in categories:
    input_category_directory = os.path.join(TESTDIR, category)
    output_category_directory = os.path.join(TESTOUT, category)

    # Ensure the output category directory exists; create it if it doesn't
    os.makedirs(output_category_directory, exist_ok=True)

    # List all files in the input category directory
    file_list = [f for f in os.listdir(input_category_directory) if f.endswith('.raw')]

    # Loop through the RF signal files in the category
    for file_name in file_list:
        # Construct the full file paths
        input_file_path = os.path.join(input_category_directory, file_name)
        output_file_path = os.path.join(output_category_directory, file_name.replace('.raw', '_stft.png'))

        # Read the RF signal data from the input file and convert it to a NumPy array
        with open(input_file_path, 'rb') as file:
            raw_data = file.read()
            rf_signal = np.frombuffer(raw_data, dtype=np.int32).astype(np.float32)

        # Calculate the STFT for the RF signal
        frequencies, times, Sxx = spectrogram(rf_signal, fs=sample_rate, nperseg=8192, noverlap=512)

        # Plot and save the STFT as a grayscale image in the output directory
        plt.figure(figsize=(5, 6))
        plt.imshow(10 * np.log10(Sxx), cmap='gray', aspect='auto', origin='lower')
        plt.colorbar(label='Power Spectral Density (dB/Hz)')
        plt.title(f'STFT of {file_name}')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        #plt.savefig(output_file_path)
        plt.imsave(output_file_path, 10 * np.log10(Sxx), cmap='gray')
        plt.close()

        print(f'Saved STFT image to {output_file_path}')

Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101707_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101713_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101719_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101725_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101731_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101737_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101743_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101749_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101755_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101801_stft.png
Saved STFT image to ./../Data/MelSpecV1/Hyundai/Lock/keyfob_signal20231122101807_stft.png
Saved STFT

In [ ]:
# WITHOUT TENSORFLOW
center_freq = 314.92e6

# List categories (e.g., Category1, Category2)
categories = ['Lock', 'Unlock', 'None']

for category in categories:
    input_category_directory = os.path.join(TOYOTALC, category)
    output_category_directory = os.path.join(LCOUT, category)

    # Ensure the output category directory exists; create it if it doesn't
    os.makedirs(output_category_directory, exist_ok=True)

    # List all files in the input category directory
    file_list = [f for f in os.listdir(input_category_directory) if f.endswith('.raw')]

    # Loop through the RF signal files in the category
    for file_name in file_list:
        # Construct the full file paths
        input_file_path = os.path.join(input_category_directory, file_name)
        output_file_path = os.path.join(output_category_directory, file_name.replace('.raw', '_stft.png'))

        # Read the RF signal data from the input file and convert it to a NumPy array
        with open(input_file_path, 'rb') as file:
            raw_data = file.read()
            rf_signal = np.frombuffer(raw_data, dtype=np.int32).astype(np.float32)

        # Set the sample rate (you need to specify the actual sample rate)
        sample_rate = 10e6  # Change this to the correct sample rate

        # Calculate the STFT for the RF signal
        frequencies, times, Sxx = spectrogram(rf_signal, fs=sample_rate, nperseg=8192, noverlap=512)

        # Plot and save the STFT as a grayscale image in the output directory
        plt.figure(figsize=(5, 6))
        plt.imshow(10 * np.log10(Sxx), cmap='gray', aspect='auto', origin='lower')
        plt.colorbar(label='Power Spectral Density (dB/Hz)')
        plt.title(f'STFT of {file_name}')
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.savefig(output_file_path)
        plt.close()

        print(f'Saved STFT image to {output_file_path}')

In [ ]:
# Directory where Mel spectrogram images are stored
data_directory = TESTOUT

# List of categories (subdirectories in the data directory)
categories = os.listdir(data_directory)

# Initialize lists to store image data and corresponding labels
X = []  # Image data
y = []  # Labels

# Loop through categories and load images
for category in categories:
    category_directory = os.path.join(data_directory, category)
    files = os.listdir(category_directory)
    for file in files:
        # Load each image and convert it to a NumPy array
        img = tf.keras.preprocessing.image.load_img(os.path.join(category_directory, file), target_size=(IMG_SIZE, IMG_L))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        X.append(img_array)
        y.append(category)  # Use category as the label

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_to_id = {class_name: class_id for class_id, class_name in enumerate(categories)}
y_train = [label_to_id[label] for label in y_train]
y_test = [label_to_id[label] for label in y_test]

# Convert labels to one-hot encoding
num_classes = len(categories)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Print the shape of the data arrays
print("X_train shape:", X_train.shape)
print("X_val shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_test.shape)

CHANGE THIS TO STAY IN BINARY FOR RNN

In [ ]:
def create_signal_dataframe(data_dir):
    I_data_list = []
    labels_list = []
    mel_spec_list = []


    for category in os.listdir(data_dir):
        category_dir = os.path.join(data_dir, category)
        if os.path.isdir(category_dir):
            for file in os.listdir(category_dir):
                if file.endswith(".raw"):
                    file_path = os.path.join(category_dir, file)

                    with open(file_path, 'rb') as file:
                        raw_data = file.read()

                    # TODO: DO NOT DO THIS for the regression training version strickly for CNN one    
                    raw_array = np.frombuffer(raw_data, dtype=np.int16).astype(np.complex64)
                    I_data = raw_array.real
                    Q_data = raw_array.imag

                    # Create a time axis for the signal (assuming a fixed sample rate)
                    sample_rate = 2e6  # Replace with your actual sample rate
                    time = np.arange(0, len(I_data)) / sample_rate

                    # Define the trim duration in seconds
                    trim_start = 0.005  # 5 ms
                    trim_end = 0.005    # 5 ms

                    # Calculate the number of samples to trim
                    trim_samples = int(sample_rate * (trim_start + trim_end))

                    # Trim the signal
                    I_trimmed = I_data[trim_samples:-trim_samples]
                    time_trimmed = time[trim_samples:-trim_samples]
                    #

                    # Pad the signal to the desired length
                    if len(I_trimmed) < 22766528:
                        pad_width = 22766528 - len(I_trimmed)
                        I_padded = np.pad(I_trimmed, (0, pad_width), 'constant')
                    else:
                        I_padded = I_trimmed


                    I_scaled = scaler.fit_transform(I_padded.reshape(-1,1))

                    I_data_list.append(I_scaled)
                    #I_data_list.append(I_trimmed)
                    # Process and pack the signals

                    
                    labels_list.append(category)
    
    #processed_signals = process_signals(I_data_list)

    one_hot_labels = pd.get_dummies(labels_list, columns=['Label'])                

    df = pd.DataFrame({'Signal': I_data_list})
    #df = pd.DataFrame({'Signal': processed_signals})
    df = pd.concat([df, one_hot_labels], axis=1)

    return df

In [ ]:
# Example usage
data_directory = TESTDIR  # Change this to the path of your 'Data' directory
#hyundai_df, melspec = create_signal_dataframe(data_directory)

In [ ]:
#hyundai_df.head()
#hyundai_df.to_csv('hyundai_data_early.csv', index=False)


In [ ]:
#hyundai_df_shuffled = hyundai_df.sample(frac=1, random_state=43).reset_index(drop=True)
#hyundai_df_shuffled.head()

In [ ]:
#X = hyundai_df_shuffled['Signal'].tolist()
#y = hyundai_df_shuffled.drop('Signal',axis=1).values

In [ ]:
"""X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=43,
)"""

In [ ]:

test = max(X_train,key=len)

In [ ]:
X_train[4].size

In [ ]:
test.size

In [ ]:
# Save the trained model
#torch.save(model.state_dict(), 'rf_signal_classifier.pth')

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
# Build the regression neural network
model = tf.keras.Sequential()
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu",input_shape=(IMG_SIZE,IMG_L,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(500, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentrophy', metrics=['accuracy'])  # Mean Squared Error (MSE) loss for regression


In [ ]:
# Train the model
model.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=3, 
    validation_data=(X_test, y_test),
    verbose=2)